# Main Idea of Finding POIs

> Notice that the POIs are *Infrastructures*, not anomaly events!

POIs are stable topological control points of the air traffic network, such as:

- Waypoints (turning points)

- Route intersections

- Merge / diverge nodes

- Entry / exit fixes

- Holding pattern anchors

## Initialization:

In [ ]:
from gensim.models import Word2Vec
import json

with open("data/flight_routes_comprehensive.geojson") as flight:
    track = json.load(flight)

sequences = []

## Data Structure Selection

### A geojson with structure (option A)

In [ ]:
Geometry
├─ X axis: [x_posi, x_velo]
├─ Y axis: [y_posi, y_velo]
└─ Z axis: [z_posi, z_velo]

In [ ]:
for feature in track["features"]:
    data_x = feature["geometry"]["axis_x"]
    tokens_x = [
        f"{lon:.3f}_{vx:.3f}" for lon, vx in data_x
    ]

for feature in track["features"]:
    data_y = feature["geometry"]["axis_y"]
    tokens_y = [
        f"{lat:.3f}_{vy:.3f}" for lat, vy in data_y
    ]

for feature in track["features"]:
    data_z = feature["geometry"]["axis_z"]
    tokens_z = [
        f"{hei:.3f}_{vz:.3f}" for hei, vz in data_z
    ]

### A geojson with structure (option B)

In [ ]:
Geometry
├─ Coords: [x_posi, y_posi, z_posi]
└─ Velocity: [x_velo, y_velo, z_velo]

In [ ]:
for feature in track["features"]:
    coords = feature["geometry"]["coords"]
    tokens_coords = [
        f"{lon:.3f}_{lat:.3f}_{hei:.3f}" for lon, lat, hei in coords
    ]

for feature in track["features"]:
    velocity = feature["geometry"]["velocity"]
    tokens_velocity = [
        f"{lon_v:.3f}_{lat_v:.3f}_{hei_v:.3f}" for lon_v, lat_v, hei_v in velocity
    ]

### Ultimate idea

In order to prevent semantic collapse, I decide to try:

In [ ]:
Embedding = f(position | velocity_context)

## Prediction Algorithm

### Position Prediction

Notably, option A suits better with 1D, option B suits better with 3D!

1. 3D interpolation route prediction
2. 1D interpolation route prediction (Merge into 3d later)
3. 3D movement physical interpolation prediction (Use velocity to predict the position and calculate momentum)
4. 1D movement physical interpolation prediction (Use velocity to predict the position and calculate momentum, Merge into 3d later)

### Velocity Prediction

My main idea is to calculate the momentum and use data to find a pattern of the velocity change!

## Thresholds

### Distance Threshold

My idea is to find a threshold of the distance to find which point is not on the expectation, using the distance below:

1. Euclidean
2. Manhattan
3. Minkowski

Once a position does not match the prediction, and above / below the threshold, we can consider this is a POI!

### Velocity Threshold

My idea is to find a threshold of the velocity changes to find which point is not on the expectation, using some sort of calculation method...

Once a velocity change does not match the prediction, and above / below the threshold, we can consider this is a POI!

-----

### Feedback from AI

Correct approach: adaptive thresholds

Use distribution-aware thresholds, not constants:

In [ ]:
z = (error - μ_local) / σ_local
POI if |z| > τ

Where μ_local, σ_local are computed:

- Per flight phase

- Per altitude band

- Or per learned cluster

## Detection Method: Graph-based POI detection

### POI as graph articulation points

In the route graph:

Remove a node <=> Graph connectivity drops

These might be:

- Mandatory turning points

- Bottlenecks

- Critical intersections

Algorithm:

- Build directed weighted graph

- Compute articulation points / high betweenness nodes

- These are designed POIs, not learned noise.

### POI as semantic transition hubs

Using embeddings:

A POI is where:

- Incoming edges come from multiple semantic clusters

- Outgoing edges diverge into multiple semantic clusters

Formally:

> POI score ∝ H(community_in) + H(community_out)

Where H is entropy.


### One More Thought

Use Transformer architecture somewhere when needed.